<a href="https://colab.research.google.com/github/Ms-Ainebyona/Text-clustering/blob/main/NER2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U spacy -q

In [ ]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.4.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines        en_core_web_sm (3.4.0)        



In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm


In [ ]:
nlp=spacy.blank("en")
db=DocBin()

In [ ]:
from google.colab import files
files.upload()

Saving cop3.json to cop3.json


{'cop3.json': b'{"classes":["GPE","DATE","CURRENCY","FIGURE","LOCATION","PERSON","ORG","PERCENTAGE","APPLICATIONS","ELECTRONICS","INSTITUITIONS","ANIMAL","FOOD","WEEK DAY","MONTH","PEOPLE","YEAR","DAY","REGION"],"annotations":[["Ugandan authorities restored access to the internet Wednesday, a month after blocking it ahead of the January 14 elections. The government said the disruption was needed for security, while critics say it was intended to cut off communication among opponents of President Yoweri Museveni.\\r",{"entities":[[52,61,"DATE"],[102,109,"WEEK DAY"],[110,112,"FIGURE"],[128,138,"ORG"],[262,278,"PERSON"]]}],["\\"Internet and social media services have been fully restored,\\" Ugandan Minister for Information and Communications Technology Peter Ogwang tweeted Wednesday, adding, \\"We apologize for the inconveniences caused, but it was for the security of our country.\\"\\r",{"entities":[[126,138,"PERSON"],[147,156,"WEEK DAY"]]}],["A tweet by Peter Ogwang, Ugandan Minister fo

In [ ]:
import json
y = open('cop3.json')
TRAIN_DATA = json.load(y)

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./cop3.spacy") # save the docbin object

100%|██████████| 148/148 [00:00<00:00, 1795.45it/s]


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./cop3.spacy --paths.dev ./cop3.spacy

ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-09-08 21:58:48,537] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-09-08 21:58:48,549] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-09-08 21:58:48,552] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-09-08 21:58:48,553] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-09-08 21:58:48,802] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  

In [ ]:
nlp_ner = spacy.load("/content/model-best")

In [ ]:
doc = nlp_ner('''There is a challenge of food in Uganda. Gloria goes to Kyambogo University. She knows programming in HTML and CSS. Prof. Twinomujuni sent the team in Isingiro some 100 USD. Students will bbe leaving the University on Friday September 20.They will graduate in 2023. Uganda has many parts that is the north, east, west and south. Uganda receives aid from countries like USA, Germany, India, China. Rose lives in Muyenga''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [ ]:
doc = nlp_ner('''Students who live in Kikoni study from CONAS''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True) 

In [ ]:
doc = nlp_ner('''Moses will finish his exams on 20th June''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True) 

In [ ]:
def evaluate(ner_model, doc):
    scorer = Scorer()
    example = []
    for input_, annot in doc:
        pred = ner_model(input_)
        print(pred,annot)
        temp = Example.from_dict(pred, dict.fromkeys(annot))
        doc.append(temp)
    scores = scorer.score(doc)
    return scores

In [ ]:
from spacy.scorer import Scorer
from spacy.training.example import Example

In [ ]:
def flat_accuracy(text, annotations):
    actual_ents = [ents[2] for ents in annotations]
    prediction = nlp_ner(text)
    pred_ents = [ent.text for ent in prediction.ents]
    return 1 if actual_ents == pred_ents else 0


predict_points = sum(flat_accuracy(test_text[0], test_text[1]) for test_text in doc)
output = (predict_points/len(dc)) * 100

TypeError: ignored